In [1]:
import pymongo as pm
import urllib
from scrape import Scrape as sc
from pprint import pprint
from bs4 import BeautifulSoup as bs
import re

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn)

In [3]:
# define db and collections
db = client.scrape_db

# this collection will store all the HTML pages
htmlCollection = db.HTML_pages

In [13]:
startURL = 'http://books.toscrape.com/'
done_urls = set()

urls = set()
pagelinks = set()

urls.add(startURL)

In [14]:
ctr = 0
while (len(urls) > 0) and (ctr < 20):
    
    for url in urls:
        print("URL in for loop: ", url)
        html = sc.scrapeHTML(url, htmlCollection, done_urls)
        if html: 
            #print(f'Html scraped: {html[:50]}')
            pagelinks = sc.scrapeURL(html, url)
            print(f'Extracted links: {len(pagelinks)}')
                  #{pagelinks}')
        
    
    # add links from page to urls set outside of for loop
    for link in pagelinks:
        urls.add(link)
    print("Total urls: ", len(urls))
    
    # remove done_urls from todo urls
    urls -= done_urls
    print("done_urls: ", done_urls)
    print("(urls - done_urls): ", len(urls))
    ctr += 1
    print("LOOP counter: ", ctr)
    print("--------------------")


URL in for loop:  http://books.toscrape.com/
<Response [200]>
Extracted links: 73
Total urls:  74
done_urls:  {'http://books.toscrape.com/'}
(urls - done_urls):  73
LOOP counter:  1
--------------------
URL in for loop:  http://books.toscrape.com/catalogue/mesaerion-the-best-science-fiction-stories-1800-1849_983/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/libertarianism-for-beginners_982/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/category/books/philosophy_7/index.html
<Response [200]>
Extracted links: 63
URL in for loop:  http://books.toscrape.com/catalogue/category/books/food-and-drink_33/index.html
<Response [200]>
Extracted links: 73
URL in for loop:  http://books.toscrape.com/catalogue/category/books/music_14/index.html
<Response [200]>
Extracted links: 65
URL in for loop:  http://books.toscrape.com/catalogue/category/books/self-help_41/index.html
<Response [200]>
Extr

<Response [200]>
Extracted links: 70
URL in for loop:  http://books.toscrape.com/catalogue/category/books/mystery_3/index.html
<Response [200]>
Extracted links: 73
URL in for loop:  http://books.toscrape.com/catalogue/category/books/classics_6/index.html
<Response [200]>
Extracted links: 71
URL in for loop:  http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html
<Response [200]>
Extracted links: 4
URL in for loop:  http://books.toscrape.com/catalogue/scott-pilgrims-precious-little-life-scott-pilgrim-1_987/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/category/books/childrens_11/index.html
<Response [200]>
Extracted links: 73
URL in for loop:  http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html
<Response [200]>
Extracted links: 3
URL in for loop:  http://books.toscrape.com/catalogue/category/books/add-a-comment_18/index.html
<Response [200]>
Extracted links: 73
URL in for loop:  http://books.toscr

<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/why-the-right-went-wrong-conservatism-from-goldwater-to-the-tea-party-and-beyond_781/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/dark-notes_800/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/amid-the-chaos_788/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/the-art-and-science-of-low-carbohydrate-living_570/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/10-day-green-smoothie-cleanse-lose-up-to-15-pounds-in-10-days_581/index.html
<Response [200]>
Extracted links: 9
Total urls:  15
done_urls:  {'http://books.toscrape.com/catalogue/mesaerion-the-best-science-fiction-stories-1800-1849_983/index.html', 'http://books.toscrape.com/catalogue/libertarianism-for-beginners_982/index.html', 'htt

<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/silence-in-the-dark-logan-point-4_542/index.html
<Response [200]>
Extracted links: 9
Total urls:  14
done_urls:  {'http://books.toscrape.com/catalogue/libertarianism-for-beginners_982/index.html', 'http://books.toscrape.com/catalogue/category/books/autobiography_27/index.html', 'http://books.toscrape.com/catalogue/category/books/biography_36/index.html', 'http://books.toscrape.com/catalogue/10-day-green-smoothie-cleanse-lose-up-to-15-pounds-in-10-days_581/index.html', 'http://books.toscrape.com/catalogue/category/books/parenting_28/index.html', 'http://books.toscrape.com/catalogue/the-black-maria_991/index.html', 'http://books.toscrape.com/catalogue/category/books/erotica_50/index.html', 'http://books.toscrape.com/catalogue/silence-in-the-dark-logan-point-4_542/index.html', 'http://books.toscrape.com/catalogue/category/books/romance_8/index.html', 'http://books.toscrape.com/catalogue/category/book

<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/you-are-a-badass-how-to-stop-doubting-your-greatness-and-start-living-an-awesome-life_508/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/overload-how-to-unplug-unwind-and-unleash-yourself-from-the-pressure-of-stress_725/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/online-marketing-for-busy-authors-a-step-by-step-guide_913/index.html
<Response [200]>
Extracted links: 9
Total urls:  13
done_urls:  {'http://books.toscrape.com/catalogue/libertarianism-for-beginners_982/index.html', 'http://books.toscrape.com/catalogue/you-are-a-badass-how-to-stop-doubting-your-greatness-and-start-living-an-awesome-life_508/index.html', 'http://books.toscrape.com/catalogue/category/books/autobiography_27/index.html', 'http://books.toscrape.com/catalogue/category/books/biography_36/index.html', 'http://books.tos

...

<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/close-to-you_798/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/32-yolks_510/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/naturally-lean-125-nourishing-gluten-free-plant-based-recipes-all-under-300-calories_479/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/redeeming-love_826/index.html
<Response [200]>
Extracted links: 9
Total urls:  12
done_urls:  {'http://books.toscrape.com/catalogue/libertarianism-for-beginners_982/index.html', 'http://books.toscrape.com/catalogue/shadows-of-the-past-logan-point-1_541/index.html', 'http://books.toscrape.com/catalogue/you-are-a-badass-how-to-stop-doubting-your-greatness-and-start-living-an-awesome-life_508/index.html', 'http://books.toscrape.com/catalogue/category/books/autobiography_27/index.html', 

<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/cravings-recipes-for-what-you-want-to-eat_589/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/the-moosewood-cookbook-recipes-from-moosewood-restaurant-ithaca-new-york_574/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/everyday-italian-125-simple-and-delicious-recipes_618/index.html
<Response [200]>
Extracted links: 9
Total urls:  13
done_urls:  {'http://books.toscrape.com/catalogue/libertarianism-for-beginners_982/index.html', 'http://books.toscrape.com/catalogue/shadows-of-the-past-logan-point-1_541/index.html', 'http://books.toscrape.com/catalogue/you-are-a-badass-how-to-stop-doubting-your-greatness-and-start-living-an-awesome-life_508/index.html', 'http://books.toscrape.com/catalogue/category/books/autobiography_27/index.html', 'http://books.toscrape.com/catalogue/category/books/biography_

<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/deliciously-ella-every-day-quick-and-easy-recipes-for-gluten-free-snacks-packed-lunches-and-simple-meals_801/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/my-kitchen-year-136-recipes-that-saved-my-life_646/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/barefoot-contessa-at-home-everyday-recipes-youll-make-over-and-over-again_666/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/the-help-yourself-cookbook-for-kids-60-easy-plant-based-recipes-kids-can-make-to-stay-healthy-and-save-the-earth_751/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/its-all-easy-healthy-delicious-weeknight-meals-in-under-30-minutes_706/index.html
<Response [200]>
Extracted links: 9
Total urls:  13
done_urls:  {'htt

<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/the-nerdy-nummies-cookbook-sweet-treats-for-the-geek-in-all-of-us_853/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/the-love-and-lemons-cookbook-an-apple-to-zucchini-celebration-of-impromptu-cooking_849/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/layered-baking-building-and-styling-spectacular-cakes_904/index.html
<Response [200]>
Extracted links: 9
Total urls:  13
done_urls:  {'http://books.toscrape.com/catalogue/libertarianism-for-beginners_982/index.html', 'http://books.toscrape.com/catalogue/shadows-of-the-past-logan-point-1_541/index.html', 'http://books.toscrape.com/catalogue/you-are-a-badass-how-to-stop-doubting-your-greatness-and-start-living-an-awesome-life_508/index.html', 'http://books.toscrape.com/catalogue/category/books/autobiography_27/index.html', 'http://books.toscrape.c

<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/foolproof-preserving-a-guide-to-small-batch-jams-jellies-pickles-condiments-and-more-a-foolproof-guide-to-making-small-batch-jams-jellies-pickles-condiments-and-more_978/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/mama-tried-traditional-italian-cooking-for-the-screwed-crude-vegan-and-tattooed_908/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/the-pioneer-woman-cooks-dinnertime-comfort-classics-freezer-food-16-minute-meals-and-other-delicious-ways-to-solve-supper_943/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/house-of-lost-worlds-dinosaurs-dynasties-and-the-story-of-life-on-earth_170/index.html
<Response [200]>
Extracted links: 9
URL in for loop:  http://books.toscrape.com/catalogue/the-guns-of-august_254/index.html
<Response [200]>